# Traffic fatalities vs. homicides, per year

In [1]:
from utz import *
from utz.colors import colors_lengthen
from utz.plots import save
import plotly.express as px
from nj_crashes.paths import HOMICIDES_PQT, PLOTS_DIR
from nj_crashes.colors import get_colors, gridcolor
from njdot.paths import CM_PQT

In [2]:
dot = read_parquet(CM_PQT)
dot

,Date,County,Total Injured,Property Damage,Total Killed
0,2001-01-01,ATLANTIC,340,494,2
1,2001-01-01,BERGEN,1053,2765,1
2,2001-01-01,BURLINGTON,347,818,6
3,2001-01-01,CAMDEN,525,957,0
4,2001-01-01,CAPE MAY,65,102,1
...,...,...,...,...,...
247,2022-12-01,SALEM,55,91,0
248,2022-12-01,SOMERSET,240,643,1
249,2022-12-01,SUSSEX,59,270,0
250,2022-12-01,UNION,459,1309,3


In [3]:
dot_yrs = dot.groupby(dot.Date.dt.year)['Total Killed'].sum()
dot_yrs

Date
2001    726
2002    609
2003    752
2004    738
2005    751
2006    745
2007    710
2008    613
2009    577
2010    567
2011    609
2012    575
2013    538
2014    551
2015    588
2016    625
2017    684
2018    622
2019    587
2020    611
2021    721
2022    732
Name: Total Killed, dtype: int64

In [4]:
homicides = read_parquet(HOMICIDES_PQT).homicides
cmp = (
    sxs(
        dot_yrs.rename('fatalities'),
        homicides
    )
    .sort_index()
    .dropna(subset='fatalities')
    .astype(int)
)
years = cmp.index
years.name = 'year'
cmp

,fatalities,homicides
year,,
2001,726,336
2002,609,339
2003,752,406
2004,738,392
2005,751,417
2006,745,428
2007,710,380
2008,613,376
2009,577,319


Average ratio, 2017-2022 <a id="avg-2017-2022"></a>

In [12]:
cmp17 = cmp[cmp.index >= 2017].sum()
cmp17.fatalities / cmp17.homicides

2.1036682615629982

In [5]:
traffic_homicide_ratio = round((cmp.fatalities / cmp.homicides).rename('ratio'), 2)
traffic_homicide_ratio

year
2001    2.16
2002    1.80
2003    1.85
2004    1.88
2005    1.80
2006    1.74
2007    1.87
2008    1.63
2009    1.81
2010    1.53
2011    1.60
2012    1.48
2013    1.33
2014    1.57
2015    1.62
2016    1.68
2017    2.11
2018    2.18
2019    2.24
2020    1.86
2021    1.90
2022    2.42
Name: ratio, dtype: float64

In [6]:
from IPython.display import Markdown, display

def swatches(colors, sep=' ', width=6):
    display(Markdown(sep.join(
        f'<span style="font-family: monospace">{color} <span style="color: {color}">{chr(9608)*width}</span></span>'
        for color in colors
    )))    

In [7]:
colors = get_colors(len(years))
black, red, year_colors = colors.black, colors.red, colors.year_colors

swatches(year_colors)

<span style="font-family: monospace">#fcffa4 <span style="color: #fcffa4">██████</span></span> <span style="font-family: monospace">#f9eb77 <span style="color: #f9eb77">██████</span></span> <span style="font-family: monospace">#f7d74b <span style="color: #f7d74b">██████</span></span> <span style="font-family: monospace">#f8c12d <span style="color: #f8c12d">██████</span></span> <span style="font-family: monospace">#f9aa15 <span style="color: #f9aa15">██████</span></span> <span style="font-family: monospace">#f9930a <span style="color: #f9930a">██████</span></span> <span style="font-family: monospace">#f37e17 <span style="color: #f37e17">██████</span></span> <span style="font-family: monospace">#ed6925 <span style="color: #ed6925">██████</span></span> <span style="font-family: monospace">#e05933 <span style="color: #e05933">██████</span></span> <span style="font-family: monospace">#d34941 <span style="color: #d34941">██████</span></span> <span style="font-family: monospace">#c33d4d <span style="color: #c33d4d">██████</span></span> <span style="font-family: monospace">#b13258 <span style="color: #b13258">██████</span></span> <span style="font-family: monospace">#9e2961 <span style="color: #9e2961">██████</span></span> <span style="font-family: monospace">#8b2267 <span style="color: #8b2267">██████</span></span> <span style="font-family: monospace">#781c6d <span style="color: #781c6d">██████</span></span> <span style="font-family: monospace">#64156c <span style="color: #64156c">██████</span></span> <span style="font-family: monospace">#500e6b <span style="color: #500e6b">██████</span></span> <span style="font-family: monospace">#3c0c5f <span style="color: #3c0c5f">██████</span></span> <span style="font-family: monospace">#280c4d <span style="color: #280c4d">██████</span></span> <span style="font-family: monospace">#170a38 <span style="color: #170a38">██████</span></span> <span style="font-family: monospace">#0b051e <span style="color: #0b051e">██████</span></span> <span style="font-family: monospace">#000004 <span style="color: #000004">██████</span></span>

In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
years = cmp.index.tolist()
y2 = 'Ratio: Crashes / Homicides'
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=cmp.index, y=cmp.fatalities, name='Traffic deaths', marker_color=red))
fig.add_trace(go.Bar(x=cmp.index, y=cmp.homicides, name='Homicides', marker_color=black))
fig.add_trace(go.Scatter(x=traffic_homicide_ratio.index, y=traffic_homicide_ratio, name=y2, line=dict(width=6, color='#f99708'), mode='lines'), secondary_y=True)
save(
    fig,
    title='NJ Traffic Deaths vs. Homicides',
    name='crash_homicide_cmp',
    dir=PLOTS_DIR,
    bg='white',
    hoverx=True,
    xaxis=dict(
        tickmode='array',
        tickvals=years,
        ticktext=years,
        tickangle=-45,
        fixedrange=True,
    ),    
    ytitle='Deaths',
    yaxis=dict(
        fixedrange=True,
        range=(0, 800),
        dtick=100,
    ),
    yaxis2=dict(
        title=y2,
        gridcolor='#ccc',
        fixedrange=True,
        range=(1, 2.6),
        dtick=.2,
    ),
    legend=dict(
        orientation="h",
        x=0.5, xanchor="center",
    ),
    png=(1100, 600),
);

Wrote plot JSON to ../www/public/plots/crash_homicide_cmp.json
Wrote plot image to ../www/public/plots/crash_homicide_cmp.png


![](../www/public/plots/crash_homicide_cmp.png)